In [9]:
from wilcoxon.spiderman import *
from wilcoxon.utils import *
from wilcoxon.sheets import *
import re
from tqdm import tqdm
import pandas as pd

In [2]:
postcodeMap = getGeo("Postcode").drop_duplicates("Postcode").set_index("Postcode").to_dict("index")

In [3]:
site = website("https://www.pa.gov.sg/our-network/community-clubs/locate-cc")

In [4]:
names = re.findall("'(.*?)'",str(site.html("script")[-10]))

In [7]:
Ls = []
for name in tqdm(names):
    name = re.sub("\s*@\s*"," ",name)
    ccSite = website("https://www.pa.gov.sg/our-network/community-clubs/locate-cc/detail/" + "-".join(name.split()))
    postcode = re.findall("\d{6}",strippedStrings(ccSite.html.find(text="Contact Information").find_next())[0])[0]
    try:
        lat = postcodeMap[int(postcode)]["Lat"]
        long = postcodeMap[int(postcode)]["Long"]
    except:
        try:
            lat,long = postcodeCoords(name)
        except:
            lat,long=["",""]
    try: ccFac = ccSite.html(text=re.compile("Facilities:"))[0].split(": ")[1]
    except: ccFac = ""
    try: vdFac = ccSite.html(text=re.compile("Facilities:"))[1].split(": ")[1]
    except: vdFac = ""
    try: ccCount = ccSite.html(text=re.compile("Counter Services:"))[0].split(": ")[1]
    except: ccCount = ""
    try: vdCount = ccSite.html(text=re.compile("Counter Services:"))[1].split(": ")[1]
    except: vdCount = ""
    L = {"Name":name,
        "Postcode":postcode,
        "Lat":lat,
        "Long":long,
        "CC Facilities":ccFac,
        "CC Counter Services":ccCount,
        "Void Deck Facilities":vdFac,
        "Void Deck Counter Services":vdCount}
    Ls.append(L)

100%|██████████| 110/110 [00:15<00:00,  6.98it/s]


In [11]:
pd.DataFrame(Ls).to_clipboard(index=False)